In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
llm = ChatOllama(model='phi3', temperature=0.0)
llm.invoke("Tell me a joke about math.")

AIMessage(content=' Why was the equal sign so humble? Because it was always too "equal" for inequality!', response_metadata={'model': 'phi3', 'created_at': '2024-07-08T15:14:02.9058381Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1877702600, 'load_duration': 1613380700, 'prompt_eval_count': 11, 'prompt_eval_duration': 36694000, 'eval_count': 21, 'eval_duration': 231488000}, id='run-cd9f1c79-96ad-4e30-b53f-9a8f404a823c-0')

In [3]:
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

- Each document is mapped to an individual summary using and LLM Chain `MapReduceDocumentsChain`.
- Then we will use `ReduceDocumentsChain` to combine those summaries together into a single summary.

In [4]:
# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes
Helpful Answer:"""

map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

c:\Users\Jigar\anaconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [5]:
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the mail themes.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

- The `ReduceDocumentsChain` handles taking mapping results and reducing them into a single output. It has the ability to collapse documents before passing it to `CombineDocumentsChain` if their size exceeds `token_max`.

- So if the cumulative number of tokens in our mapped documents exceeds 4000 tokens, then we'll recursively pass in the documents in batches of < 4000 tokens to our `StuffDocumentsChain` to create batched summaries. And once those batched summaries are cumulatively less than 4000 tokens, we'll pass them all one last time to the `StuffDocumentsChain` to create the final summary.

In [6]:
# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is the final chain that is called
    combine_documents_chain = combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain = combine_documents_chain,
    # The maximum number of tokens to group document into.
    token_max=4000
)

In [14]:
# Combining the map and reduce chains into one
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # THe variable name in the llm chain to put the documents in
    document_variable_name='docs',
    return_intermediate_steps=True,
    verbose=True
)

In [10]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 1003, which is longer than the specified 1000


In [15]:
result = map_reduce_chain.invoke(split_docs)

print(result["output_text"])



> Entering new MapReduceDocumentsChain chain...


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (4948 > 1024). Running this sequence through the model will result in indexing errors



> Finished chain.
 The primary theme across the documents is centered on developing efficient workflows for creating complex software projects like Super Mario games using Python MVC components with keyboard control inputs as well as leveraging Large Language Models (LLMs) to create autonomous agents capable of performing tasks or making decisions in various applications, including scientific research and interactive simulations.

Key aspects discussed include:
- Designing a software architecture for the Super Mario game using MVC components with clear definitions/comment blocks within core classes representing levels, characters, movement mechanics, obstacles, enemies, etc., to facilitate readability by future developers.
- Organizing Python codes into separate files based on functionality and creating an entrypoint file that imports necessary modules as needed for the main program flow (e.g., `main.py`, `game_logic.py`).
- Writing tests using pytest, covering all possible scenarios 

In [18]:
result

{'input_documents': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content="LLM Powered Autonomous Agents | Lil'Log\n\nLil'Log\n\n\nPosts\n\n\nArchive\n\n\nSearch\n\n\nTags\n\n\nFAQ\n\n\nemojisearch.app\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\nAgent System O